In [0]:
import asyncio
import numpy as np
from PIL import Image
import time
from concurrent import futures


In [0]:
def main():
    async def f1():
        await asyncio.sleep(0.1)

    async def f2(im, i):
        for j in range(60):
            Image.fromarray(im).convert("L").save("./{}.png".format(str(i)))

    async def f3(im, i):
        for _ in range(60):
            g = Image.fromarray(im).convert("L")
            g.save("./{}.png".format(str(i)))

    im = np.random.randint(0, 256, (240, 256, 3)).astype("uint8")

    loop = asyncio.get_event_loop()
    s = time.time()
    for i in range(10):
        result = loop.run_until_complete(
            asyncio.gather(
                f1(),
                f3(im, i)
            )
        )
    print(time.time() - s)
    
main()

3.782989263534546


In [0]:
def main2():
    async def f1():
        await asyncio.sleep(0.1)

    async def f3(im, i):
        for _ in range(60):
            g = Image.fromarray(im).convert("L")
            g.save("./{}.png".format(str(i)))

    im = np.random.randint(0, 256, (240, 256, 3)).astype("uint8")

    loop = asyncio.get_event_loop()
    s = time.time()
    for i in range(10):
        result = loop.run_until_complete(
            asyncio.gather(
                f1(),
                f3(im, i)
            )
        )
    print(time.time() - s)
    
main2()

2.7405879497528076


In [0]:
def main3():
    def f1():
        time.sleep(0.1)
        return 1

    def f3(im, i):
        for _ in range(60):
            g = Image.fromarray(im).convert("L")
            g.save("./{}.png".format(str(i)))
        return 3

    im = np.random.randint(0, 256, (240, 256, 3)).astype("uint8")

    loop = asyncio.get_event_loop()
    executor = futures.ThreadPoolExecutor(max_workers=10)

    s = time.time()

    for i in range(10):
        coroutine_list = [loop.run_in_executor(executor, f3, *(im,i)),
                  loop.run_in_executor(executor, f1, )]
        task = asyncio.gather(*coroutine_list)
        task = asyncio.ensure_future(task)
        loop.run_until_complete(task)
        # print(task.result())

    print(time.time() - s)


main3()



2.7594215869903564
